In [ ]:
# https://dzone.com/articles/exporting-data-from-pdfs-with-python
# simple pdfminer
# http://survivalengineer.blogspot.com/2014/04/parsing-pdfs-in-python.html
# https://github.com/pdfminer/pdfminer.six

In [18]:
from fpdf import FPDF
import PyPDF2
from PIL import Image

import sys
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from io import StringIO
from pdfminer.layout import LAParams
from pdfminer.converter import TextConverter


In [11]:
pdf = FPDF()
imagelist = ['pdf1_to_read.jpg']
for image in imagelist:
    pdf.add_page()
    pdf.image(image,0,0,200,250)
pdf.output("real_pdf.pdf", "F")

''

In [19]:
pdfFileObj = open('4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
pages = pdfReader.numPages
pageObj = pdfReader.getPage(0)
text = pageObj.extractText()
pages # text

9

In [31]:
import io
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage
def extract_text_by_page(pdf_path):
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, 
                                      caching=True,
                                      check_extractable=True):
            resource_manager = PDFResourceManager()
            fake_file_handle = io.StringIO()
            converter = TextConverter(resource_manager, fake_file_handle)
            page_interpreter = PDFPageInterpreter(resource_manager, converter)
            page_interpreter.process_page(page)
            text = fake_file_handle.getvalue()
            yield text
            # close open handles
            converter.close()
            fake_file_handle.close()
def extract_text(pdf_path):
    for page in extract_text_by_page(pdf_path):
        print(page)
        print()

In [34]:
import io
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage
def extract_text_from_pdf(pdf_path):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle)
    page_interpreter = PDFPageInterpreter(resource_manager, converter)
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, 
                                      caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)
        text = fake_file_handle.getvalue()
    # close open handles
    converter.close()
    fake_file_handle.close()
    if text:
        return text

In [35]:
import json
import os
# from miner_text_generator import extract_text_by_page

def export_as_json(pdf_path, json_path):
    filename = os.path.splitext(os.path.basename(pdf_path))[0]
    data = {'Filename': filename}
    data['Pages'] = []
    counter = 1
    for page in extract_text_by_page(pdf_path):
        text = page[0:100]
        page = {'Page_{}'.format(counter): text}
        data['Pages'].append(page)
        counter += 1
    with open(json_path, 'w') as fh:
        json.dump(data, fh)

In [38]:
extract_text_from_pdf('00213193_PDF.pdf')

'018786 PDI0183618A9TT50I0183618A9TI0183618A9TH01836249XX01836249HS018787 PDI01836189TDA0183618B9TI01836199TDA0183619A9TI0183619B9TI50I0183619A9TH0183619A9TT50I0183622A9TH0183622A9TI0183622A9TT0183622B9TI01836229TDA0183615A9TH0183615A9TI0183615A9TT50I0183615B9TI01836159TDA0183621B9TI01836219TDA50 I 0183621A9TI0183621A9TH0183621A9TT01836239TDA0183623A9TT0183623A9TH50 I 0183623A9TI0183623B9TI018793 TW018788 PDI018794 TW018789  PDII018 HS50 I6464 2131946720213321N 6740 2131946741 213286MDO6742 213286 MDO6743 2132026744 213286 MDO 6745 213286 MDO 6739 213195DO NOT POCKET SELF DRAININGRSSELF DRAINING DO NOT POCKET RSMIN.. 1DELETED.. 2HIGH HIGH LEVEL IN SUCTION DRUMS AND SIGNAL FROMCOMPRESSOR AUXILARY CAUSES COMPRESSOR SHUTDOWN.. 3FOR COMPRESSOR AUXILIARIES, SEE DRAWING YC-213357TO YC-213380.. 4N2 CONNECTION FOR COMPRESSOR PURGING.. 5DETAILS OF COMPRESSOR CAPACITY CONTROL SYSTEM TO BEAS PER COMPRESSOR VENDOR DATA.. 6ALL LINE NUMBERS, SPECIALTY ITEMS (SP) AND NON-DCS(LOCAL) INSTRUMENT TAG NUM

In [39]:
export_as_json('00213193_PDF.pdf', '00213193_PDF.json')